### Explore Batter Data

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',None)

In [2]:
batter_profiles = pd.read_csv('milb_batter_summaries.csv')
batter_seasons = pd.read_csv('milb_all_batter_seasons.csv')

In [3]:
len(batter_profiles.milb_id.unique())

17525

__Helper Functions__

In [4]:
def add_season_number(df):
    first_year = pd.DataFrame()
    first_year['First_Year'] = df.groupby(['milb_id']).min()['Year']
    first_year = first_year.reset_index()
    df = df.merge(first_year, how = 'outer')
    
    df['season_number'] = df['Year'] - df['First_Year'] + 1
    #df = df.drop('First_Year',axis = 1)
    return df


def add_success_year(df):
    success_year = pd.DataFrame()
    success_year['First_Year'] = df.groupby(['milb_id']).min()['Year']
    first_year = first_year.reset_index()
    df = df.merge(first_year, how = 'outer')
    
    df['season_number'] = df['Year'] - df['First_Year'] + 1
    #df = df.drop('First_Year',axis = 1)
    return df

def add_season_number(df):
    first_year = pd.DataFrame()
    first_year['First_Year'] = df.groupby(['milb_id']).min()['Year']
    first_year = first_year.reset_index()
    df = df.merge(first_year, how = 'outer')
    
    df['season_number'] = df['Year'] - df['First_Year'] + 1
    #df = df.drop('First_Year',axis = 1)
    return df



# Calculate the number of single from other hit data
def add_singles(df):
    df['1B'] = df['H'] - df['2B'] - df['3B'] - df['HR']
    return df


__Filtering and Aggregation__

In [5]:
batter_seasons = add_season_number(batter_seasons)

In [6]:
# Remove Players Who had their first season before 1990.
#batter_seasons

# Map Each League to a level
lev_dict = {'FRk':0,'Rk':1,'A-':2,'A':3,'A+':4,'AA':5,'AAA':6}



# Leagues I don't care about



In [10]:
majors_df = batter_seasons[batter_seasons.Lev == 'MLB'][['milb_id','Year']]

In [11]:
mlb_debuts = majors_df.groupby('milb_id').min()

In [12]:
batter_seasons_filt = batter_seasons.merge(mlb_debuts, how = 'outer')

MemoryError: 

In [ ]:
# 'CREATE TABLE IF NOT EXISTS BatterSeasons (
#     Year int DEFAULT NULL,
#     Age int DEFAULT NULL,
#     AgeDif float DEFAULT NULL,
#     G int DEFAULT NULL,
#     PA int DEFAULT NULL,
#     AB int DEFAULT NULL,
#     R int DEFAULT NULL,
#     H int DEFAULT NULL,
#     2B int DEFAULT NULL,
#     3B int DEFAULT NULL,
#     HR int DEFAULT NULL,
#     RBI int DEFAULT NULL,
#     SB int DEFAULT NULL,
#     CS int DEFAULT NULL,
#     BB int DEFAULT NULL,
#     SO int DEFAULT NULL,
#     BA float DEFAULT NULL,
#     OBP float DEFAULT NULL,
#     SLG float DEFAULT NULL,
#     OPS float DEFAULT NULL,
#     TB int DEFAULT NULL,
#     GDP int DEFAULT NULL,
#     HBP int DEFAULT NULL,
#     SH int DEFAULT NULL,
#     SF int DEFAULT NULL,
#     IBB int DEFAULT NULL,
#     Tm varchar(20) NOT NULL,
#     Lg varchar(20) NOT NULL,
#     Lev varchar(20) NOT NULL,
#     Aff varchar(20) NOT NULL,
#     mlb_id varchar(20) NOT NULL,
#     milb_id varchar(20) NOT NULL],
    
    
    
    
    
    
#     playerID varchar(20) NOT NULL,
#     yearID int NOT NULL,
#     gameNum varchar(20) NOT NULL,
#     gameID varchar(12) DEFAULT NULL,
#     teamID text DEFAULT NULL,
#     lgID text DEFAULT NULL,
#     GP varchar(20) DEFAULT NULL,
#     startingPos varchar(20) DEFAULT NULL,
#     PRIMARY KEY (playerID,yearID,gameNum)
# );'